In [1]:
import json
from os.path import join
import os
os.chdir("/home/tiagoalmeida/bioASQ-taskb/")

import sys
import pickle
import gc
import numpy as np
import tarfile
import random
from collections import defaultdict
from bisect import bisect
import math

##add keras to the modules
module_path = os.path.abspath(os.path.join('pubmed_data'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pubmed_data import pubmed_helper as ph

EMB_DIM = 200

import


## HAR

Reference PAPER:http://dmkd.cs.vt.edu/papers/WWW19.pdf

### Network structure
 - [General Network Configuration](#var_def)
 - [Self Attention Layer](#self_attn)
 - [Cross Attention Layer](#cross_attn)
 - [Final Network](#final_net)

In [2]:

#Load tokenizer and the embedding matrix

MODE = "regex_full_tokens"
tk = ph.load_tokenizer(mode=MODE)
emb_dict = ph.load_embeddings(mode=MODE)

assert len(tk.word_counts) == len(emb_dict)

#Number of different words
VOCAB_SIZE = len(tk.word_counts)+1

#Dimension of embeddings
EMB_DIM = emb_dict[1].shape[0]

emb_matrix = np.zeros((VOCAB_SIZE, EMB_DIM))

for i,vector in emb_dict.items():
    emb_matrix[i] = vector

Load regex_full_tokens_tokenizer.p
Load regex_full_tokens_word_embedding.p


<a id='var_def'></a>
## General Network Configuration

In [3]:

from tensorflow import unstack, stack
##Test 
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, activations
from tensorflow.keras.initializers import Zeros, Ones, Constant
from tensorflow.keras.layers import Dropout,Dense, Lambda, Bidirectional, Dot,Masking,Reshape, Concatenate, Layer, Embedding, Input, Conv2D, GlobalMaxPooling2D, Flatten, TimeDistributed, GRU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.activations import tanh, sigmoid

from tensorflow.keras.preprocessing.sequence import pad_sequences


K.clear_session()

#Number max of term per query
MAX_Q_TERM = 13

#Number max of the snippet terms
MAX_SNIPPET_LENGTH = 13

MAX_NUMBER_SNIPPETS = 25

MAX_DOCUMENT_TOKENS = MAX_SNIPPET_LENGTH*MAX_NUMBER_SNIPPETS

ATTENTION_DIMENSION = 100

assert ATTENTION_DIMENSION%2==0

GRU_DIM = ATTENTION_DIMENSION//2

NUM_OF_SELF_ATTENTION = 1
#Train embedding weights
EMB_TRAINABLE = False

ACTIVATION_FUNCTION = "selu"




<a id='self_attn'></a>
### Self Attention Layer

In [4]:
class MaskedSelfAttention(Layer):
    
    def __init__(self, attention_dimension=ATTENTION_DIMENSION, initializer='glorot_normal', regularizer=None, **kargs):
        super(MaskedSelfAttention, self).__init__(**kargs)

        self.initializer = initializer
        self.attention_dimension = attention_dimension
        
        self.num_of_self_attention = 1
        self.attention_weights = []
        
        if regularizer is None or isinstance(regularizer,str):
            self.regularizer = regularizers.get(regularizer)
        else:
            self.regularizer = regularizer

    def build(self, input_shape):
        
        emb_dim = int(input_shape[2])

        self.W_attn_project = self.add_variable(name = "self_attention_projection",
                                   shape = [emb_dim, self.attention_dimension],
                                   initializer = self.initializer,
                                   regularizer = self.regularizer,)
        
        self.W_attn_score = self.add_variable(name = "self_attention_score",
                                   shape = [self.attention_dimension, self.num_of_self_attention],
                                   initializer = self.initializer,
                                   regularizer = self.regularizer,)

        super(MaskedSelfAttention, self).build(input_shape)
    
    def call(self, x):
        
        #x_transpose = K.permute_dimensions(x,[0,2,1]) # (NONE, 100,15)
        #print("x_transpose",x_transpose)
        condition = K.all(x,keepdims=True,axis=-1)
        print("condition",condition)
        inv_condition = (1-K.cast(condition, K.floatx()))
        print("inv_condition",inv_condition)
        
        x_projection = K.dot(x, self.W_attn_project) # (NONE, 300, 300)
        print("x_projection",x_projection)
        x_tanh = K.tanh(x_projection) # (NONE, 300, 15)
        print("x_tanh",x_tanh)
        x_attention = K.dot(x_tanh, self.W_attn_score)
        print("x_attention",x_attention)
        x_attention_maked = x_attention + (1.0 - inv_condition) * -10000.0
        print("x_attention_maked",x_attention_maked)
        #x_attention_squeeze = K.squeeze(x_score, axis=1)
        x_attention_softmax = K.softmax(x_attention_maked,axis = 1)
        print("x_attention_softmax",x_attention_softmax)
        
        if (self.num_of_self_attention>1):
            x_attention_softmax_transpose =  K.permute_dimensions(x_attention_softmax,[0,2,1])
            print("x_attention_softmax_transpose",x_attention_softmax_transpose)

            x_attention_softmax_transpose_expand = K.expand_dims(x_attention_softmax_transpose)
            print("x_attention_softmax_transpose_expand",x_attention_softmax_transpose_expand)

            x_scored_emb = x_attention_softmax_transpose_expand * x
            print("x_scored_emb",x_scored_emb)
            x_attention_rep = K.sum(x_scored_emb, axis=2)
        else:
            x_scored_emb = x_attention_softmax * x
            print("x_scored_emb",x_scored_emb)
            x_attention_rep = K.sum(x_scored_emb, axis=1)
        
        #save attent w
        self.attention_weights.append(x_attention_softmax)
        
        print("x_attention_rep",x_attention_rep)
        return x_attention_rep


<a id='cross_attn'></a>
### Cross Attention Layer

In [5]:
class CrossAttention(Layer):
    
    def __init__(self, initializer='glorot_normal', regularizer=None, **kargs):
        super(CrossAttention, self).__init__(**kargs)

        self.initializer = initializer
        
        if regularizer is None or isinstance(regularizer,str):
            self.regularizer = regularizers.get(regularizer)
        else:
            self.regularizer = regularizer

    def build(self, input_shape):
        
        """
        input: [0] - query context embedding
               [1] - document context embedding
        """
        doc_embedding = input_shape[1]
        query_embedding = input_shape[0]
        
        self.query_len = query_embedding[1]
        self.doc_len = doc_embedding[1]
        
        print("query_len", self.query_len)
        print("doc_len", self.doc_len)
        
        assert int(query_embedding[2]) == int(doc_embedding[2])
        
        self.embedding_dim = int(query_embedding[2])
        
        self.W_sim_projection = self.add_variable(name = "similarity_projection",
                                   shape = [self.embedding_dim*3,1],
                                   initializer = self.initializer,
                                   regularizer = self.regularizer,)
        
        super(CrossAttention, self).build(input_shape)
    
    def call(self, x):
        """
        input: [0] - query context embedding
               [1] - document context embedding
        """
        doc_embedding = x[1]
        print("doc_embedding",doc_embedding.shape)
        query_embedding = x[0]
        print("query_embedding",query_embedding.shape)
        
        #build similarity matrix
        #row document token
        #colum query token
        doc_q_matrix = K.expand_dims(doc_embedding, axis=2)
        print("doc_q_matrix",doc_q_matrix.shape)
        doc_q_matrix = K.repeat_elements(doc_q_matrix, self.query_len, axis=2)
        print("doc_q_matrix",doc_q_matrix.shape)
        
        q_doc_matrix = K.expand_dims(query_embedding, axis=1)
        print("q_doc_matrix",q_doc_matrix.shape)
        q_doc_matrix = K.repeat_elements(q_doc_matrix, self.doc_len, axis=1)
        print("q_doc_matrix",q_doc_matrix.shape)
        
        element_mult = doc_q_matrix * q_doc_matrix
        print("element_mult",element_mult.shape)
        
        #concatenation
        S = K.concatenate([doc_q_matrix, q_doc_matrix, element_mult])
        print("S",S.shape)
        print("Wc",self.W_sim_projection.shape)
        S = K.dot(S, self.W_sim_projection)
        print("S",S.shape)
        
        S = K.squeeze(S,axis=-1)
        print("S",S.shape)
        
        S_D2Q = K.softmax(S, axis=1)
        print("S_D2Q",S_D2Q.shape)
        
        S_Q2D = K.softmax(S, axis=2)
        print("S_Q2D",S_Q2D.shape)
        
        A_D2Q = K.batch_dot(S_D2Q, query_embedding)
        print("A_D2Q",A_D2Q.shape)
        
        S_Q2D_transpose = K.permute_dimensions(S_Q2D,[0,2,1])
        print("S_Q2D_transpose",S_Q2D_transpose.shape)
        
        A_D2Q_Q2D = K.batch_dot(S_D2Q, S_Q2D_transpose)
        print("A_D2Q_Q2D",A_D2Q_Q2D.shape)
        
        A_Q2D = K.batch_dot(A_D2Q_Q2D, doc_embedding)
        print("A_Q2D",A_Q2D.shape)
        
        #concat
        doc_attn = doc_embedding * A_D2Q
        print("doc_attn",doc_attn.shape)
        doc_q_attn = doc_embedding * A_Q2D
        print("doc_q_attn",doc_q_attn.shape)
        
        V = K.concatenate([doc_embedding, A_D2Q, doc_attn, doc_q_attn])
        
        return V

In [6]:
#Test
visualize_cross_attention = False
if visualize_cross_attention:
    input_doc = Input(shape=(MAX_Q_TERM,))
    input_query = Input(shape=(MAX_SNIPPET_LENGTH,))

    emb_layer = Embedding(1000,300)

    cross_layer = CrossAttention()

    emb_doc = emb_layer(input_doc)
    emb_query = emb_layer(input_query)

    cross_attention = cross_layer([emb_query, emb_doc])

    model = Model(inputs=[input_query,input_doc], outputs = [cross_attention])
    model.summary()

<a id='final_net'></a>
### Final Network

In [7]:
REG = 0.001

#Layer Defenition

input_query = Input(shape=(MAX_Q_TERM,), name = "query_input")
input_doc = Input(shape=(MAX_NUMBER_SNIPPETS, MAX_SNIPPET_LENGTH), name = "document_input")
#input_doc = [Input(shape=(MAX_SNIPPET_LENGTH,), name = "document_input_"+j) for j in range(MAX_NUMBER_SNIPPETS)]

unstack_snippets = Lambda(lambda x:unstack(x,axis=1), name="unstack_snippets")

input_docs = unstack_snippets(input_doc)

emb_layer = Embedding(VOCAB_SIZE,EMB_DIM, weights=[emb_matrix],name="embedding_layer", trainable=EMB_TRAINABLE)

context_encoder = Bidirectional(GRU(GRU_DIM, return_sequences=True))

query_self_attention = MaskedSelfAttention()
cross_layer = CrossAttention()
snippet_lvl_1_self_attention = MaskedSelfAttention()
snippet_lvl_2_self_attention = MaskedSelfAttention()

snippet_expand_dim = Lambda(lambda x:K.expand_dims(x,axis=1))
snippet_concat = Lambda(lambda x:K.concatenate(x,axis=1))

snippets_projection = Dense(ATTENTION_DIMENSION)

q_doc_mult = Lambda(lambda x:x[0]*x[1])

#NO ACTIVATION??
fnn_h1 = Dense(ATTENTION_DIMENSION)
fnn_h2 = Dense(ATTENTION_DIMENSION//2)
fnn_h3 = Dense(1)

"""
ASSEMBLE
"""

query_emb = emb_layer(input_query)
query_context_aware = context_encoder(query_emb)
query_weighted = query_self_attention(query_context_aware)

snippets = []
for input_snippet in input_docs:
    snippet_emb = emb_layer(input_snippet)
    snippet_context_aware = context_encoder(snippet_emb)
    snippet_query = cross_layer([query_context_aware, snippet_context_aware])
    snippet_weighted = snippet_lvl_1_self_attention(snippet_query)
    snippets.append(snippet_expand_dim(snippet_weighted))

snippets = snippet_concat(snippets)
snippets = snippet_lvl_2_self_attention(snippets)
snippets = snippets_projection(snippets)
q_snippets_rep = q_doc_mult([query_weighted, snippets])
score = fnn_h1(q_snippets_rep)
score = fnn_h2(score)
score = fnn_h3(score)

inputs = [input_query, input_doc]

har_model = Model(inputs=inputs, outputs = [score])

har_model.summary()

condition Tensor("masked_self_attention/All:0", shape=(?, 13, 1), dtype=bool)
inv_condition Tensor("masked_self_attention/sub:0", shape=(?, 13, 1), dtype=float32)
x_projection Tensor("masked_self_attention/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("masked_self_attention/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("masked_self_attention/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_maked Tensor("masked_self_attention/add:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("masked_self_attention/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("masked_self_attention/mul_1:0", shape=(?, 13, 100), dtype=float32)
x_attention_rep Tensor("masked_self_attention/Sum:0", shape=(?, 100), dtype=float32)
query_len 13
doc_len 13
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?,

doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
condition Tensor("masked_self_attention_1_6/All:0", shape=(?, 13, 1), dtype=bool)
inv_condition Tensor("masked_self_attention_1_6/sub:0", shape=(?, 13, 1), dtype=float32)
x_projection Tensor("masked_self_attention_1_6/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("masked_self_attention_1_6/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("masked_self_attention_1_6/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_maked Tensor("masked_self_attention_1_6/add:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor

doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
condition Tensor("masked_self_attention_1_13/All:0", shape=(?, 13, 1), dtype=bool)
inv_condition Tensor("masked_self_attention_1_13/sub:0", shape=(?, 13, 1), dtype=float32)
x_projection Tensor("masked_self_attention_1_13/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("masked_self_attention_1_13/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("masked_self_attention_1_13/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_maked Tensor("masked_self_attention_1_13/add:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax 

doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
condition Tensor("masked_self_attention_1_20/All:0", shape=(?, 13, 1), dtype=bool)
inv_condition Tensor("masked_self_attention_1_20/sub:0", shape=(?, 13, 1), dtype=float32)
x_projection Tensor("masked_self_attention_1_20/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("masked_self_attention_1_20/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("masked_self_attention_1_20/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_maked Tensor("masked_self_attention_1_20/add:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax 

## Build training Arch

In [8]:
# total number of negative samples

#input

input_query = Input(shape=(MAX_Q_TERM,))
input_positive_doc = Input(shape=(MAX_NUMBER_SNIPPETS, MAX_SNIPPET_LENGTH,))
input_negative_doc = Input(shape=(MAX_NUMBER_SNIPPETS, MAX_SNIPPET_LENGTH,))

positive_score = har_model([input_query, input_positive_doc])
negative_score = har_model([input_query, input_negative_doc])

har_trainable_model = Model(inputs=[input_query, input_positive_doc, input_negative_doc], outputs=[positive_score, negative_score])

#write tensor operation loss function

p_loss = K.mean(K.maximum(0.0, 1.0 - positive_score + negative_score))
    
har_trainable_model.add_loss(p_loss)


har_trainable_model.summary()



doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix 

S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)


x_projection Tensor("model/masked_self_attention_1_8/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("model/masked_self_attention_1_8/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("model/masked_self_attention_1_8/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_maked Tensor("model/masked_self_attention_1_8/add:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("model/masked_self_attention_1_8/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model/masked_self_attention_1_8/mul_1:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model/masked_self_attention_1_8/Sum:0", shape=(?, 400), dtype=float32)
condition Tensor("model/masked_self_attention_1_9/All:0", shape=(?, 13, 1), dtype=bool)
inv_condition Tensor("model/masked_self_attention_1_9/sub:0", shape=(?, 13, 1), dtype=float32)
x_projection Tensor("model/masked_self_attention_1_9/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("model/m

x_projection Tensor("model/masked_self_attention_1_18/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("model/masked_self_attention_1_18/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("model/masked_self_attention_1_18/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_maked Tensor("model/masked_self_attention_1_18/add:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("model/masked_self_attention_1_18/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model/masked_self_attention_1_18/mul_1:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model/masked_self_attention_1_18/Sum:0", shape=(?, 400), dtype=float32)
condition Tensor("model/masked_self_attention_1_19/All:0", shape=(?, 13, 1), dtype=bool)
inv_condition Tensor("model/masked_self_attention_1_19/sub:0", shape=(?, 13, 1), dtype=float32)
x_projection Tensor("model/masked_self_attention_1_19/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tenso

S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)


S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_

x_attention_softmax Tensor("model_1/masked_self_attention_1_8/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model_1/masked_self_attention_1_8/mul_1:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model_1/masked_self_attention_1_8/Sum:0", shape=(?, 400), dtype=float32)
condition Tensor("model_1/masked_self_attention_1_9/All:0", shape=(?, 13, 1), dtype=bool)
inv_condition Tensor("model_1/masked_self_attention_1_9/sub:0", shape=(?, 13, 1), dtype=float32)
x_projection Tensor("model_1/masked_self_attention_1_9/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("model_1/masked_self_attention_1_9/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("model_1/masked_self_attention_1_9/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_maked Tensor("model_1/masked_self_attention_1_9/add:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("model_1/masked_self_attention_1_9/transpose_3:0", shape=(?, 13, 1), dtype=floa

x_attention_softmax Tensor("model_1/masked_self_attention_1_18/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model_1/masked_self_attention_1_18/mul_1:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model_1/masked_self_attention_1_18/Sum:0", shape=(?, 400), dtype=float32)
condition Tensor("model_1/masked_self_attention_1_19/All:0", shape=(?, 13, 1), dtype=bool)
inv_condition Tensor("model_1/masked_self_attention_1_19/sub:0", shape=(?, 13, 1), dtype=float32)
x_projection Tensor("model_1/masked_self_attention_1_19/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("model_1/masked_self_attention_1_19/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("model_1/masked_self_attention_1_19/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_maked Tensor("model_1/masked_self_attention_1_19/add:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("model_1/masked_self_attention_1_19/transpose_3:0", shape=(?, 13, 1), 

In [9]:
WRITE_GRAPH = False
if WRITE_GRAPH:
    from tensorflow.summary import FileWriter

    graph = K.get_session().graph
     # Your model implementation
    #with graph.as_default():
      # compile method actually creates the model in the graph.
      #deepRank_model.compile(loss=identity_loss, optimizer='adam', metrics=['accuracy'])
    writer = FileWriter(logdir='tensorboard/har', graph=graph)
    writer.flush()



## Set up training data

In [10]:
path_dl_train = "/backup/results/fast_method_relevant_results/train_data_deep_models_v2.tar.gz"



tar = tarfile.open(path_dl_train)
#open
print("Open",path_dl_train)
m = tar.getmembers()[0]
f = tar.extractfile(m)
train_articles_collection = pickle.load(f)

f.close()

Open /backup/results/fast_method_relevant_results/train_data_deep_models_v2.tar.gz


In [11]:
PARTILLY_POSITIVE_SAMPLES = 2
NEGATIVE_SAMPLES = 3

class TrainDataGenerator(object):
    def __init__(self, article_collection, tokenizer, batch_queries_size):
        
        self.batch_size = batch_queries_size
        self.tokenizer = tokenizer
        
        self.train_data = article_collection["bioasq_data"]
        self.articles = article_collection["collection"]
        self.irrelevant_pmid = article_collection["irrelevant_pmid"]
        
        self.num_steps = len(self.train_data)//self.batch_size
    
    def __len__(self):
        return self.num_steps
    
    def __iter__(self):
        
        
        query = []
        query_positive_doc = [] 
        query_negative_doc = []

        
        while True:
            
            #stop condition
            if len(query)>=self.batch_size:
                #missing fill the gap for the missing query_terms
                query = np.array(query)
                p=np.random.permutation(query.shape[0])
                query = query[p]
                query_positive_doc = np.array(query_positive_doc)[p]
                query_negative_doc = np.array(query_negative_doc)[p]
                
                X = [query, query_positive_doc, query_negative_doc]
                #Y = [np.zeros((len(query))),np.zeros((len(query)))]
                yield X


                #reset
                query = []
                query_positive_doc = [] 
                query_negative_doc = []
            

            #select a random question
            random_query_index = random.randint(0, len(self.train_data)-1) 
            query_data = self.train_data[random_query_index]
            
            #list of partilly relevant documents
            partilly_positive_pmid_docs = query_data["partilly_positive_pmid"]
            tokenized_query = query_data["query"][:MAX_Q_TERM]
            
            
            
            #positive doc
            #select a random positive
            
            
            
            for j in range(PARTILLY_POSITIVE_SAMPLES+NEGATIVE_SAMPLES):
                random_doc_index = random.randint(0, len(query_data["positive_pmid"])-1) 
                doc_pmid = query_data["positive_pmid"][random_doc_index]

                tokenized_positive_doc = self.articles[doc_pmid]
                positive_snippets = self.__snippet_split(tokenized_positive_doc)
                
                if j<PARTILLY_POSITIVE_SAMPLES:
                    #select the partilly posivite doc
                    random_ind = bisect(query_data["partially_positive_cumulative_prob"],random.random())
                    random_negative_doc_pmid = query_data["partilly_positive_pmid"][random_ind]
                    #print(self.__get_article(random_negative_doc_pmid))
                    tokenized_negative_doc = self.articles[random_negative_doc_pmid]
                    negative_snippets = self.__snippet_split( tokenized_negative_doc)
                else:
                    #select a random negative
                    random_doc_index = random.randint(0, len(self.irrelevant_pmid)-1) 
                    doc_pmid = self.irrelevant_pmid[random_doc_index]
                    
                    tokenized_negative_doc = self.articles[doc_pmid]
                    negative_snippets = self.__snippet_split( tokenized_negative_doc)
                
                #negative doc
                query.append(tokenized_query)
                query_positive_doc.append(positive_snippets)
                query_negative_doc.append(negative_snippets)



            
    def __snippet_split(self, tokenized_doc, snippet_length=MAX_SNIPPET_LENGTH):
        

        if len(tokenized_doc) < MAX_DOCUMENT_TOKENS:
            #pad
            tokenized_doc += [0]*(MAX_DOCUMENT_TOKENS-len(tokenized_doc))
            
        else:
            tokenized_doc = tokenized_doc[:MAX_DOCUMENT_TOKENS] #cut
        
        index_list = list(range(0, MAX_DOCUMENT_TOKENS, MAX_SNIPPET_LENGTH))+[MAX_DOCUMENT_TOKENS]
        #print(tokenized_doc)
        snippets = [ tokenized_doc[index_list[i]:index_list[i+1]] for i in range(len(index_list)-1)  ]
        #print(snippets)
        return snippets
        
        

In [12]:
data_generator = TrainDataGenerator(train_articles_collection, tk, 128)
data_generator = iter(data_generator)
X = next(data_generator)

In [13]:
X[0].shape

(135, 13)

## Test (validation) data

In [12]:
path_dl_test = "/backup/results/fast_method_relevant_results/test_data_deep_models_v2.tar.gz"



tar = tarfile.open(path_dl_test)
#open
print("Open",path_dl_test)
m = tar.getmembers()[0]
f = tar.extractfile(m)
test_articles_collection = pickle.load(f)

f.close()

Open /backup/results/fast_method_relevant_results/test_data_deep_models_v2.tar.gz


In [13]:
class TestDataGenerator(object):
    def __init__(self, article_collection, tokenizer):
        
        self.tokenizer = tokenizer
        
        self.test_data = article_collection["bioasq_data"] 
        self.articles = article_collection["collection"]
        
        self.num_steps = len(self.test_data)
        
    
    def __get_article(self, pmid):
        return self.article_map(self.articles[pmid])
    
    def __len__(self):
        return self.num_steps
    
    def __iter__(self):
        
        
        query = []
        query_doc = []
        

        for query_data in self.test_data:


            tokenized_query = query_data["query"][:MAX_Q_TERM]

            for doc_pmid in query_data["documents"]:

                tokenized_doc = self.articles[doc_pmid]
                doc_snippets = self.__snippet_split(tokenized_doc)

                ### add ###
                query.append(tokenized_query)

                #positive doc
                query_doc.append(doc_snippets)


            #missing fill the gap for the missing query_terms

            X = [np.array(query), np.array(query_doc)]

            yield X

            #reset
            query = []
            query_doc = []

                
    def __snippet_split(self, tokenized_doc, snippet_length=MAX_SNIPPET_LENGTH):
        

        if len(tokenized_doc) < MAX_DOCUMENT_TOKENS:
            #pad
            tokenized_doc += [0]*(MAX_DOCUMENT_TOKENS-len(tokenized_doc))
            
        else:
            tokenized_doc = tokenized_doc[:MAX_DOCUMENT_TOKENS] #cut
        
        index_list = list(range(0, MAX_DOCUMENT_TOKENS, MAX_SNIPPET_LENGTH))+[MAX_DOCUMENT_TOKENS]
        #print(tokenized_doc)
        snippets = [ tokenized_doc[index_list[i]:index_list[i+1]] for i in range(len(index_list)-1)  ]
        #print(snippets)
        return snippets

In [14]:
validation_articles_collection = {"bioasq_data":[],"collection":test_articles_collection["collection"]}

_temp = test_articles_collection["bioasq_data"][:]
random.shuffle(_temp)
print(len(_temp))

validation_percentage = 0.15

split_index = int(len(_temp)*validation_percentage)
print(split_index)

validation_articles_collection["bioasq_data"] = _temp[:split_index]
#test_articles_collection["bioasq_data"] = _temp[split_index:]

print("validation size",len(validation_articles_collection["bioasq_data"]),"test size",len(test_articles_collection["bioasq_data"]))

549
82
validation size 82 test size 549


In [15]:
def validate_test_data(data):
    data_generator = TestDataGenerator(data, tk)
    data_generator = iter(data_generator)

    query_results = {}

    for i,X in enumerate(data_generator):
        print("Predict query:",i,end="\r")
        har_ranking = har_model.predict(X)
        har_ranking = map(lambda x:x[0],har_ranking.tolist())
        bm25_results = data["bioasq_data"][i]["documents"]
        har_ranking_pmid = list(zip(bm25_results,har_ranking))
        har_ranking_pmid.sort(key=lambda x:-x[1])
        query_results[data["bioasq_data"][i]["id"]] = {"result":har_ranking_pmid,"goldstandard":data["bioasq_data"][i]["positive_pmid"]}
        #print("save query results:",i,end="\r")
        
    return query_results

In [16]:
from models.generic_model import ModelAPI, f_recall, f_map

def validation_score(deep_rank_test_query_results):
    id_to_remove = []
    for k,v in deep_rank_test_query_results.items():
        if len(v["goldstandard"]) == 0:
            id_to_remove.append(k)

    for k in id_to_remove:
        del deep_rank_test_query_results[k]

    deep_rank_test_query_results = list(deep_rank_test_query_results.values())

    print("TEST set, len ",len(deep_rank_test_query_results))

    expectations = list(map(lambda x:x["goldstandard"],deep_rank_test_query_results))
    predictions = list(map(lambda x:x["result"],deep_rank_test_query_results))

    #print("Recall:",f_recall(predictions,expectations,at=1000))
    bioasq_map = f_map(predictions,expectations,bioASQ=True)
    print("MAP @10 bioASQ:", bioasq_map)
    print("MAP @25:",f_map(predictions,expectations, bioASQ=True, at=25))
    print("MAP @50:",f_map(predictions,expectations, bioASQ=True, at=50))
    print("MAP @100:",f_map(predictions,expectations, bioASQ=True, at=100))
    print("MAP @200:",f_map(predictions,expectations, bioASQ=True, at=200))
    print("MAP @300:",f_map(predictions,expectations, bioASQ=True, at=300))

    print("MAP:",f_map(predictions,expectations, use_len=True))
    
    print("RECALL@10:",f_recall(predictions,expectations, at=10))
    print("RECALL@50:",f_recall(predictions,expectations, at=50))
    print("RECALL@100:",f_recall(predictions,expectations, at=100))
    return bioasq_map, predictions, expectations

In [17]:
from tensorflow.keras.optimizers import SGD, Adam,  Adadelta

#sgd = SGD(lr=0.001)
#adam = Adam(lr=0.001)
adadelta = Adadelta(lr=2)

har_trainable_model.compile(optimizer=adadelta)

In [20]:
#validation = validate_test_data(validation_articles_collection)

In [21]:
#_ = validation_score(validation)

In [18]:
gen = TrainDataGenerator(train_articles_collection, tk, 256)

gen_iter = iter(gen)

loss = []

for i,line in enumerate(loss):
    print("Epoach:",i,"| avg loss:",np.mean(loss[i]),"| max loss:",np.max(loss[i]),"| min loss:",np.min(loss[i]))

import time

max_bio_map_val = 0.08
max_bio_map_test = 0

for epoach in range(0,1000):
    loss_per_epoach = []
    for step in range(len(gen)):
        X = next(gen_iter)
        
        start = time.time()
        loss_per_epoach.append(har_trainable_model.train_on_batch(X))
        print("Step:",step,"| loss:",loss_per_epoach[-1],"| current max loss:",np.max(loss_per_epoach),"| current min loss:",np.min(loss_per_epoach),"| time:",time.time()-start,end="\r")
        

    
    
    if epoach%10==0:
        print("")
        validate_query_results = validate_test_data(validation_articles_collection)
        print("")
        bio_map_val,_,_ = validation_score(validate_query_results)
        
        if bio_map_val >= max_bio_map_val:
            max_bio_map_val = bio_map_val
            print("")
            print("Run for the test set")
            test_query_results = validate_test_data(test_articles_collection)
            bio_map_test = validation_score(test_query_results)
            
            if bio_map_test >= max_bio_map_test:
                max_bio_map_test = bio_map_test
                
                har_model.save_weights("har_weights_v2_1.h5")
                #har_model.save("har_model.h5")
                
    loss.append(loss_per_epoach)
    print("",end="\r")#clear the line
    print("Epoach:",epoach,"| avg loss:",np.mean(loss[-1]),"| max loss:",np.max(loss[-1]),"| min loss:",np.min(loss[-1]))

Step: 7 | loss: 0.9999068 | current max loss: 1.0000179 | current min loss: 0.9998466 | time: 4.0773279666900635184
Predict query: 81
TEST set, len  79
MAP @10 bioASQ: 0.004641350210970465
MAP @25: 0.006070655932943393
MAP @50: 0.007180333511534399
MAP @100: 0.008957088757646515
MAP @200: 0.011835962089100304
MAP @300: 0.013311629837215512
MAP: 0.018925127286509552
RECALL@10: 0.03703079934288114
RECALL@50: 0.08925442310106156
RECALL@100: 0.1646975411274574
Epoach: 0 | avg loss: 0.9999656 | max loss: 1.0000179 | min loss: 0.9998466
Epoach: 1 | avg loss: 0.9997522 | max loss: 0.99988616 | min loss: 0.999530260.99953026 | time: 4.07522320747375596
Epoach: 2 | avg loss: 0.9994706 | max loss: 0.9997447 | min loss: 0.9991375 0.9991375 | time: 4.098817110061645552
Epoach: 3 | avg loss: 0.9992447 | max loss: 0.9998226 | min loss: 0.9990457 0.9990457 | time: 4.12006115913391189
Epoach: 4 | avg loss: 0.9986829 | max loss: 0.99925464 | min loss: 0.99808854 0.99808854 | time: 4.139498472213745
Epo

Epoach: 51 | avg loss: 0.46948433 | max loss: 0.51760983 | min loss: 0.34422597.34422597 | time: 4.101299285888672
Epoach: 52 | avg loss: 0.431451 | max loss: 0.4714445 | min loss: 0.3976956 0.3976956 | time: 4.077629089355469836
Epoach: 53 | avg loss: 0.44243973 | max loss: 0.5139876 | min loss: 0.35943228.35943228 | time: 4.0946240425109865
Epoach: 54 | avg loss: 0.42492357 | max loss: 0.5054831 | min loss: 0.3363514.3363514 | time: 4.082793951034546525
Epoach: 55 | avg loss: 0.4461637 | max loss: 0.50252485 | min loss: 0.417214040.41721404 | time: 4.100287199020386
Epoach: 56 | avg loss: 0.4738482 | max loss: 0.50607985 | min loss: 0.406682: 0.406682 | time: 4.091760635375977055
Epoach: 57 | avg loss: 0.4054633 | max loss: 0.47596794 | min loss: 0.374825330.37482533 | time: 4.119643449783325
Epoach: 58 | avg loss: 0.42678285 | max loss: 0.52278763 | min loss: 0.34420475.34420475 | time: 4.075857162475586
Epoach: 59 | avg loss: 0.4287005 | max loss: 0.51542574 | min loss: 0.33950310.

Epoach: 103 | avg loss: 0.32563487 | max loss: 0.38209397 | min loss: 0.2590981725909817 | time: 4.1154434680938725
Epoach: 104 | avg loss: 0.36174148 | max loss: 0.39712748 | min loss: 0.319688861968886 | time: 4.10048675537109445
Epoach: 105 | avg loss: 0.3765058 | max loss: 0.4281846 | min loss: 0.3175946831759468 | time: 4.1132142543792725
Epoach: 106 | avg loss: 0.35587472 | max loss: 0.4235112 | min loss: 0.3099613530996135 | time: 4.0876524448394775
Epoach: 107 | avg loss: 0.33411694 | max loss: 0.39640218 | min loss: 0.2224225.2224225 | time: 4.1280879974365232
Epoach: 108 | avg loss: 0.35930446 | max loss: 0.4122464 | min loss: 0.3085161.3085161 | time: 4.0506255626678476
Epoach: 109 | avg loss: 0.34274554 | max loss: 0.395785 | min loss: 0.2703157427031574 | time: 4.093558549880981464
Step: 7 | loss: 0.3477659 | current max loss: 0.38112137 | current min loss: 0.26191828 | time: 4.1010317802429275
Predict query: 81
TEST set, len  79
MAP @10 bioASQ: 0.01292646172393008
MAP @25

Epoach: 155 | avg loss: 0.26395836 | max loss: 0.28350163 | min loss: 0.2079780820797808 | time: 4.119190216064453
Epoach: 156 | avg loss: 0.2430537 | max loss: 0.27793616 | min loss: 0.18441603.18441603 | time: 4.062939882278442
Epoach: 157 | avg loss: 0.2553147 | max loss: 0.32257822 | min loss: 0.20040652.20040652 | time: 4.1239666938781745
Epoach: 158 | avg loss: 0.2597445 | max loss: 0.29651287 | min loss: 0.22125617.22125617 | time: 4.075632095336914
Epoach: 159 | avg loss: 0.28044274 | max loss: 0.36386847 | min loss: 0.2064179.2064179 | time: 4.09612393379211455
Step: 7 | loss: 0.21968067 | current max loss: 0.31333923 | current min loss: 0.21968067 | time: 4.1052703857421875
Predict query: 81
TEST set, len  79
MAP @10 bioASQ: 0.019561482820976493
MAP @25: 0.028411934942643366
MAP @50: 0.03503393965875059
MAP @100: 0.04086206551658496
MAP @200: 0.04648665894316095
MAP @300: 0.049362077159853274
MAP: 0.0675690803788441
RECALL@10: 0.11787680729219703
RECALL@50: 0.2751071296963166

Epoach: 207 | avg loss: 0.18882485 | max loss: 0.22223069 | min loss: 0.1588965415889654 | time: 4.1062886714935375
Epoach: 208 | avg loss: 0.23944803 | max loss: 0.27511638 | min loss: 0.2107782721077827 | time: 4.105455636978149
Epoach: 209 | avg loss: 0.21721336 | max loss: 0.27301624 | min loss: 0.1646326516463265 | time: 4.0687520503997855
Step: 7 | loss: 0.21847059 | current max loss: 0.28381935 | current min loss: 0.17132047 | time: 4.159021854400635
Predict query: 81
TEST set, len  79
MAP @10 bioASQ: 0.02117992766726944
MAP @25: 0.028487925682636455
MAP @50: 0.03632395291393341
MAP @100: 0.04273943532208728
MAP @200: 0.04890766220184554
MAP @300: 0.05172810113449359
MAP: 0.07096074284365425
RECALL@10: 0.14487484034952391
RECALL@50: 0.2933562170304343
RECALL@100: 0.39067004298408
Epoach: 210 | avg loss: 0.22186157 | max loss: 0.28381935 | min loss: 0.17132047
Epoach: 211 | avg loss: 0.21164265 | max loss: 0.26479593 | min loss: 0.1827032118270321 | time: 4.108653306961067
Epoach

...

Epoach: 774 | avg loss: 0.09124847 | max loss: 0.13648131 | min loss: 0.054065596054065596 | time: 4.266596794128418
Epoach: 775 | avg loss: 0.091855 | max loss: 0.12004835 | min loss: 0.069142275069142275 | time: 4.225766897201538795
Epoach: 776 | avg loss: 0.09137751 | max loss: 0.113289215 | min loss: 0.069313956931395 | time: 4.2782027721405038555
Epoach: 777 | avg loss: 0.10120107 | max loss: 0.12364607 | min loss: 0.0707128350712835 | time: 4.338714361190796254
Epoach: 778 | avg loss: 0.08905544 | max loss: 0.113870926 | min loss: 0.05841402758414027 | time: 4.25225019454956055
Epoach: 779 | avg loss: 0.08471174 | max loss: 0.11421078 | min loss: 0.04767946547679465 | time: 4.27060961723327615
Step: 7 | loss: 0.11895309 | current max loss: 0.1239281 | current min loss: 0.04482202 | time: 4.2687051296234135545
Predict query: 81
TEST set, len  79
MAP @10 bioASQ: 0.028629696604380153
MAP @25: 0.04360591256498121
MAP @50: 0.05474641610134555
MAP @100: 0.06514759054074212
MAP @200: 0.

Epoach: 828 | avg loss: 0.08374919 | max loss: 0.1386939 | min loss: 0.029909384029909384 | time: 4.28991079330444352
Epoach: 829 | avg loss: 0.1034825 | max loss: 0.16145101 | min loss: 0.07063896.07063896 | time: 4.25719451904296949
Step: 7 | loss: 0.09028787 | current max loss: 0.09028787 | current min loss: 0.052345548 | time: 4.28484058380126953
Predict query: 81
TEST set, len  79
MAP @10 bioASQ: 0.03802290536467751
MAP @25: 0.05259372251054349
MAP @50: 0.06419299611728917
MAP @100: 0.07758495788541131
MAP @200: 0.08522731067245953
MAP @300: 0.08866301621414557
MAP: 0.11137983026615451
RECALL@10: 0.16442253670463847
RECALL@50: 0.3552256064073549
RECALL@100: 0.5007648594827433
Epoach: 830 | avg loss: 0.071782246 | max loss: 0.09028787 | min loss: 0.052345548
Epoach: 831 | avg loss: 0.07454332 | max loss: 0.12457827 | min loss: 0.048787322048787322 | time: 4.2648916244506845
Epoach: 832 | avg loss: 0.085895106 | max loss: 0.10214218 | min loss: 0.071704817170481 | time: 4.2650475502

Epoach: 879 | avg loss: 0.09060924 | max loss: 0.11115711 | min loss: 0.0661717206617172 | time: 4.2949967384338382
Step: 7 | loss: 0.09782501 | current max loss: 0.10653385 | current min loss: 0.045554925 | time: 4.2692699432373057
Predict query: 81
TEST set, len  79
MAP @10 bioASQ: 0.030668575447056465
MAP @25: 0.041287288409302125
MAP @50: 0.05572225380136664
MAP @100: 0.06640344881190628
MAP @200: 0.07486934474043017
MAP @300: 0.07782685748888592
MAP: 0.10935125701558071
RECALL@10: 0.1914081009220481
RECALL@50: 0.3374944562400751
RECALL@100: 0.4874651740728704
Epoach: 880 | avg loss: 0.0815984 | max loss: 0.10653385 | min loss: 0.045554925
Epoach: 881 | avg loss: 0.07399909 | max loss: 0.12413201 | min loss: 0.026990745026990745 | time: 4.30039501190185555
Epoach: 882 | avg loss: 0.08195156 | max loss: 0.10878492 | min loss: 0.06082761.06082761 | time: 4.252315521240234
Epoach: 883 | avg loss: 0.08275116 | max loss: 0.13569283 | min loss: 0.057937745057937745 | time: 4.218713521957

Predict query: 81
TEST set, len  79
MAP @10 bioASQ: 0.02935101466747036
MAP @25: 0.04661473734289386
MAP @50: 0.06090507362457503
MAP @100: 0.07123705870539494
MAP @200: 0.07964323292841045
MAP @300: 0.08317291516530728
MAP: 0.09645466914394445
RECALL@10: 0.15577289003034156
RECALL@50: 0.3713517515258384
RECALL@100: 0.5154735544285266
Epoach: 930 | avg loss: 0.07890006 | max loss: 0.10067346 | min loss: 0.054458745
Epoach: 931 | avg loss: 0.08670677 | max loss: 0.1476649 | min loss: 0.0597360205973602 | time: 4.2822000980377211255
Epoach: 932 | avg loss: 0.06942578 | max loss: 0.08803508 | min loss: 0.042615954261595 | time: 4.29072570800781251
Epoach: 933 | avg loss: 0.07862552 | max loss: 0.11088073 | min loss: 0.04262947742629477 | time: 4.305668354034424525
Epoach: 934 | avg loss: 0.082703285 | max loss: 0.12851304 | min loss: 0.04342843243428432 | time: 4.3972177505493163
Epoach: 935 | avg loss: 0.08696149 | max loss: 0.113783956 | min loss: 0.033943545033943545 | time: 4.28341126

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:
validation_train_articles_collection = {"bioasq_data":[],"collection":train_articles_collection["collection"]}

_temp = train_articles_collection["bioasq_data"][:]
random.shuffle(_temp)
print(len(_temp))

validation_percentage = 0.15

split_index = int(len(_temp)*validation_percentage)
print(split_index)

validation_train_articles_collection["bioasq_data"] = _temp[:split_index]
#test_articles_collection["bioasq_data"] = _temp[split_index:]

print("validation train size",len(validation_train_articles_collection["bioasq_data"]))

del _temp

2137
85
validation train size 85


In [24]:
test = validate_test_data(test_articles_collection)
_test_validation_scores = validation_score(test)

TEST set, len  542
MAP @10 bioASQ: 0.027976117261172603
MAP @25: 0.04049144965792221
MAP @50: 0.052029734465929414
MAP @100: 0.06388542771095526
MAP @200: 0.07421527203727145
MAP @300: 0.07934577525085477
MAP: 0.048542443394001204
RECALL@10: 0.101177666088062
RECALL@50: 0.2834401857648032
RECALL@100: 0.42046242344803586


In [19]:
_train = validate_test_data(train_articles_collection)
_train_validation_scores = validation_score(_train)

TEST set, len  2137
MAP @10 bioASQ: 0.09287287103267441
MAP @25: 0.13294063845135978
MAP @50: 0.16379397029146908
MAP @100: 0.186566475002723
MAP @200: 0.200292324759265
MAP @300: 0.20470002917866395
MAP: 0.1662106613799084
RECALL@10: 0.3267852685546388
RECALL@50: 0.6615555193042019
RECALL@100: 0.809399175373454


In [40]:
har_model.save_weights("har_model_weights.h5")